In [29]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import supervisely_lib as sly

from tqdm import tqdm
from collections import defaultdict

%matplotlib inline

In [30]:
address = 'http://192.168.1.69:5555'
token = 'YGPDnuBkhFmcQ7VNzSEjhgavjg4eFR4Eq1C3jIY4HgV3SQq2JgkXCNtgZy1Fu2ftd4IKui8DsjrdtXjB853cMtBevpSJqFDYiaG1A5qphlH6fFiYYmcVZ5fMR8dDrt5l'
team_name = 'dima'
workspace_name = 'work'

src_project_name = 'cls_test'

In [31]:
api = sly.Api(address, token)

team_id = api.team.get_info_by_name(team_name)['id']
workspace_id = api.workspace.get_info_by_name(workspace_name, team_id)['id']

src_project_id = api.project.get_info_by_name(src_project_name, workspace_id)['id']

src_meta_json = api.project.get_meta(src_project_id)
src_meta = sly.ProjectMeta.from_json(src_meta_json)

In [32]:
tags_mapping = {
    "HumanTrue": "Human",
    "Human": "NoneTag"
}

In [33]:
TRUE_POSITIVE = 'true-positive'
TRUE_NEGATIVE = 'true-negative'
FALSE_POSITIVE = 'false-positive'
FALSE_NEGATIVE = 'false-negative'

METRIC_ACCURACY = 'accuracy'
METRIC_PRECISION = 'precision'
METRIC_RECALL = 'recall'
METRIC_F1_MEASURE = 'F1-measure'
METRICS_TOTAL = 'total'


def classification_metrics(ann):
    current_metric_res = {}
    for cls_1, cls_2 in tags_mapping.items():
        c1 = ann.img_tags.has_key(cls_1)
        c2 = ann.img_tags.has_key(cls_2)

        current_metric_res[(cls_1, cls_2)] = {
            TRUE_POSITIVE: int(c1 and c2),
            TRUE_NEGATIVE: int(not c1 and not c2),
            FALSE_POSITIVE: int(not c1 and c2),
            FALSE_NEGATIVE: int(c1 and not c2)
        }
    return current_metric_res


def process(ann, metric_res, metric_total_res):    
    res = classification_metrics(ann)
    for pair_name, met_data in res.items():
        for metric_name, metric_value in met_data.items():
            metric_res[pair_name][metric_name] += metric_value
            metric_total_res[metric_name] += metric_value

In [34]:
metric_res = defaultdict(lambda: {TRUE_POSITIVE: 0,
                                  TRUE_NEGATIVE: 0,
                                  FALSE_POSITIVE: 0,
                                  FALSE_NEGATIVE: 0})

metric_total_res = defaultdict(lambda: 0)

for dataset_info in api.dataset.get_list(src_project_id):
    src_dataset_id = dataset_info['id']
    src_dataset_name = dataset_info['name']

    print('Project/Dataset: {}/{}'.format(src_project_name, src_dataset_name))
    
    for image_info in tqdm(api.image.get_list(src_dataset_id)):
        src_image_ext = image_info['meta']['mime'].split('/')[1]

        ann_json = api.annotation.download(src_dataset_id, image_info['id'])
        ann = sly.Annotation.from_json(ann_json, src_meta)
        process(ann, metric_res, metric_total_res)

100%|██████████| 4/4 [00:00<00:00, 34.36it/s]

Project/Dataset: cls_test/ds


In [35]:
def calculate_complex_metrics(values, pair_name):
    TP = values[TRUE_POSITIVE]
    TN = values[TRUE_NEGATIVE]
    FP = values[FALSE_POSITIVE]
    FN = values[FALSE_NEGATIVE]

    values[METRIC_ACCURACY] = (TP + TN) / (TP + TN + FP + FN)

    def zero_division_message(metric_name):
        print('Metric "{}" for pair {} is 0, because denominator is 0'.format(metric_name, pair_name))

    try:
        values[METRIC_PRECISION] = TP / (TP + FP)
    except ZeroDivisionError:
        values[METRIC_PRECISION] = 0
        zero_division_message(METRIC_PRECISION)

    try:
        values[METRIC_RECALL] = TP / (TP + FN)
    except ZeroDivisionError:
        values[METRIC_RECALL] = 0
        zero_division_message(METRIC_RECALL)

    try:
        values[METRIC_F1_MEASURE] = (2.0 * TP) / (2.0 * TP + FP + FN)
    except ZeroDivisionError:
        values[METRIC_F1_MEASURE] = 0
        zero_division_message(METRIC_F1_MEASURE)
        
for pair_name, values in metric_res.items():
    calculate_complex_metrics(values, pair_name)
calculate_complex_metrics(metric_total_res, "(ALL, ALL)")
metric_res[METRICS_TOTAL] = metric_total_res

In [40]:
def print_evaluation_values(values):
    labels = ['accuracy', 'precision', 'recall', 'F1-measure',
              'true-positive', 'true-negative', 'false-positive', 'false-negative']
    for label in labels:
        print('    {0}:   {1:2.4f}'.format(label.ljust(16), values[label]))

print('Total values:')
print_evaluation_values(metric_res[METRICS_TOTAL])

counter = 0
for key, value in metric_res.items():
    if key == METRICS_TOTAL:
        continue
    counter += 1
    print('{0}) {1}:'.format(counter, key))
    print_evaluation_values(value)
    print()

Total values:
    accuracy        :   0.3750
    precision       :   0.6000
    recall          :   0.5000
    F1-measure      :   0.5455
    true-positive   :   3.0000
    true-negative   :   0.0000
    false-positive  :   2.0000
    false-negative  :   3.0000
1) ('HumanTrue', 'Human'):
    accuracy        :   0.5000
    precision       :   0.6667
    recall          :   0.6667
    F1-measure      :   0.6667
    true-positive   :   2.0000
    true-negative   :   0.0000
    false-positive  :   1.0000
    false-negative  :   1.0000

2) ('Human', 'NoneTag'):
    accuracy        :   0.2500
    precision       :   0.5000
    recall          :   0.3333
    F1-measure      :   0.4000
    true-positive   :   1.0000
    true-negative   :   0.0000
    false-positive  :   1.0000
    false-negative  :   2.0000

